In [ ]:
!pip install -q torch torchvision tqdm

import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
import math
from pathlib import Path

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [ ]:
# CELL 2 — configuration / hyperparameters
IMG_SIZE = 32            # CIFAR-10 native size
PATCH_SIZE = 4           # 32 / 4 = 8 -> 8x8 patches = 64 patches
EMBED_DIM = 384          # match your DALLE-inspired dim (keep if GPU can handle)
NUM_HEADS = 6
NUM_LAYERS = 6
MLP_RATIO = 4
DROPOUT = 0.1

NUM_CLASSES = 10
BATCH_SIZE = 256         # reduce if OOM
EPOCHS = 60
LR = 3e-4
WEIGHT_DECAY = 0.05
SAVE_PATH = "dalle_cifar10_best.pth"

# Mixed precision toggle
USE_AMP = True

print(f"Config: IMG={IMG_SIZE}, PATCH={PATCH_SIZE}, EMBED={EMBED_DIM}, HEADS={NUM_HEADS}, LAYERS={NUM_LAYERS}")


Config: IMG=32, PATCH=4, EMBED=384, HEADS=6, LAYERS=6


In [ ]:
# CELL 3 — model (DALLE-style Vision Transformer adapted for CIFAR-10)
# This follows the same architecture pattern in your project. See train_new.py for reference. :contentReference[oaicite:1]{index=1}

class PatchEmbedding(nn.Module):
    def __init__(self, img_size=32, patch_size=4, in_channels=3, embed_dim=384):
        super().__init__()
        assert img_size % patch_size == 0, "img_size must be divisible by patch_size"
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x)                      # (B, embed_dim, H/ps, W/ps)
        x = x.flatten(2).transpose(1, 2)      # (B, num_patches, embed_dim)
        return x


class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim=384, num_heads=6, dropout=0.0):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        assert embed_dim % num_heads == 0
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5
        self.qkv = nn.Linear(embed_dim, embed_dim * 3)
        self.proj = nn.Linear(embed_dim, embed_dim)
        self.attn_drop = nn.Dropout(dropout)
        self.proj_drop = nn.Dropout(dropout)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, self.head_dim).permute(2,0,3,1,4)
        q, k, v = qkv[0], qkv[1], qkv[2]                     # each: (B, heads, N, head_dim)
        attn = (q @ k.transpose(-2, -1)) * self.scale        # (B, heads, N, N)
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
        x = (attn @ v).transpose(1,2).reshape(B, N, C)       # (B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class MLP(nn.Module):
    def __init__(self, embed_dim=384, mlp_ratio=4, dropout=0.0):
        super().__init__()
        hidden = int(embed_dim * mlp_ratio)
        self.fc1 = nn.Linear(embed_dim, hidden)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden, embed_dim)
        self.drop = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class TransformerBlock(nn.Module):
    def __init__(self, embed_dim=384, num_heads=6, mlp_ratio=4, dropout=0.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = MultiHeadAttention(embed_dim, num_heads, dropout)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.mlp = MLP(embed_dim, mlp_ratio, dropout)

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x


class DALLEInspiredViT(nn.Module):
    def __init__(self, img_size=32, patch_size=4, embed_dim=384, num_heads=6, num_layers=6,
                 mlp_ratio=4, dropout=0.0, num_classes=10):
        super().__init__()
        num_patches = (img_size // patch_size) ** 2
        self.patch_embed = PatchEmbedding(img_size, patch_size, 3, embed_dim)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.pos_drop = nn.Dropout(dropout)
        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, mlp_ratio, dropout)
            for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes)

        # weight init similar to train_new.py
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        nn.init.trunc_normal_(self.cls_token, std=0.02)

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)
        cls = self.cls_token.expand(B, -1, -1)
        x = torch.cat([cls, x], dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)
        return self.head(x[:, 0])


In [ ]:
# CELL 4 — data loaders for CIFAR-10
# Use CIFAR-10 stats
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Download dataset (will download to /root/.cache by default)
train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=train_transform)
test_dataset  = datasets.CIFAR10(root="./data", train=False, download=True, transform=test_transform)

# Create small validation split from training set
VAL_SPLIT = 5000
train_size = len(train_dataset) - VAL_SPLIT
train_set, val_set = torch.utils.data.random_split(train_dataset, [train_size, VAL_SPLIT])

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_set,   batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_dataset,batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

print("Train/Val/Test sizes:", len(train_set), len(val_set), len(test_dataset))


100%|██████████| 170M/170M [00:24<00:00, 6.88MB/s]


Train/Val/Test sizes: 45000 5000 10000


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# CELL 5 — training helpers (train/validate)
model = DALLEInspiredViT(
    img_size=IMG_SIZE, patch_size=PATCH_SIZE,
    embed_dim=EMBED_DIM, num_heads=NUM_HEADS,
    num_layers=NUM_LAYERS, mlp_ratio=MLP_RATIO,
    dropout=DROPOUT, num_classes=NUM_CLASSES
).to(device)

print("Params:", sum(p.numel() for p in model.parameters()))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

scaler = torch.cuda.amp.GradScaler() if (USE_AMP and device.type == "cuda") else None

def train_one_epoch(epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [train]")
    for imgs, labels in pbar:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        if scaler:
            with torch.cuda.amp.autocast():
                outputs = model(imgs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()
        pbar.set_postfix(loss=running_loss/total, acc=100*correct/total)
    return running_loss/total, 100*correct/total

@torch.no_grad()
def evaluate(loader, stage="val"):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    for imgs, labels in tqdm(loader, desc=f"[{stage}]"):
        imgs, labels = imgs.to(device), labels.to(device)
        if scaler:
            with torch.cuda.amp.autocast():
                outputs = model(imgs)
                loss = criterion(outputs, labels)
        else:
            outputs = model(imgs)
            loss = criterion(outputs, labels)
        total_loss += loss.item() * imgs.size(0)
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()
    return total_loss / total, 100*correct/total


Params: 10695562


/tmp/ipython-input-1550233627.py:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if (USE_AMP and device.type == "cuda") else None


In [ ]:
# CELL 6 — training loop and checkpointing
best_val_acc = 0.0
history = {"train_loss":[], "train_acc":[], "val_loss":[], "val_acc":[]}

for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(epoch)
    val_loss, val_acc = evaluate(val_loader, stage="val")
    scheduler.step()

    history["train_loss"].append(train_loss)
    history["train_acc"].append(train_acc)
    history["val_loss"].append(val_loss)
    history["val_acc"].append(val_acc)

    print(f"Epoch {epoch+1}: train_loss={train_loss:.4f} train_acc={train_acc:.2f}% | val_loss={val_loss:.4f} val_acc={val_acc:.2f}%")

    # save best
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "epoch": epoch,
            "val_acc": val_acc
        }, SAVE_PATH)
        print(f"[SAVED] New best val_acc: {best_val_acc:.2f}% -> {SAVE_PATH}")

# final test using best checkpoint
ckpt = torch.load(SAVE_PATH, map_location=device)
model.load_state_dict(ckpt["model_state_dict"])
test_loss, test_acc = evaluate(test_loader, stage="test")
print(f"Test Accuracy (best checkpoint): {test_acc:.2f}%")


Epoch 1/60 [train]:   0%|          | 0/176 [00:00<?, ?it/s]/tmp/ipython-input-1550233627.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
[val]:   0%|          | 0/20 [00:00<?, ?it/s]/tmp/ipython-input-1550233627.py:58: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
[val]: 100%|██████████| 20/20 [00:02<00:00,  9.21it/s]


Epoch 1: train_loss=1.8418 train_acc=30.93% | val_loss=1.8570 val_acc=32.34%
[SAVED] New best val_acc: 32.34% -> dalle_cifar10_best.pth


[val]: 100%|██████████| 20/20 [00:02<00:00,  9.20it/s]


Epoch 2: train_loss=1.5478 train_acc=43.06% | val_loss=1.4413 val_acc=47.10%
[SAVED] New best val_acc: 47.10% -> dalle_cifar10_best.pth


[val]: 100%|██████████| 20/20 [00:02<00:00,  8.93it/s]


...

Epoch 59: train_loss=0.1983 train_acc=92.94% | val_loss=0.7957 val_acc=79.26%


[val]: 100%|██████████| 20/20 [00:02<00:00,  6.76it/s]


Epoch 60: train_loss=0.1930 train_acc=93.21% | val_loss=0.7851 val_acc=79.90%
[SAVED] New best val_acc: 79.90% -> dalle_cifar10_best.pth


[test]: 100%|██████████| 40/40 [00:02<00:00, 15.29it/s]

Test Accuracy (best checkpoint): 80.44%


In [ ]:
# CELL 7 — optionally: save history and download model
# If in Colab, you can download the file or move to Drive.
from google.colab import files
files.download(SAVE_PATH)    # prompts download in browser (Colab)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>